# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), True)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# You sit North and hold:
hand = 'J.32.KJ983.QJT74'

auction = ["PAD_START","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, True)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

bid X value 0.6942 is recommended by NN
CandidateBid(bid=X   , insta_score=0.6942, expected_score=---, adjust=---)
CandidateBid(bid=PASS, insta_score=0.3016, expected_score=---, adjust=---)
Sampling for aution: ['PAD_START', '2C'] trying to find 5000
sample_cards_auction, nsteps= 1
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', '2C']
nesw_i 0
Player:  S Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 5.69  4.49 21.7 ]]
Shape:  [[4.04 3.67 2.62 2.64 3.81 3.41 2.86 2.83 4.11 3.59 2.4  2.77]]
Setting seed (Sampling)= 1383891728
sample_cards_vec generating 5000
sample_cards_vec took 0.09396
n_samples 469 matching bidding info
Samples after bidding filtering:  469  Threshold:  0.25
 X    0.694 Samples: 200
bidding_rollout - n_samples:  200
bidding_rollout - finished  (200, 64)
J.xx.KJ98x.QJTxx K98x.Txxx.xxx.9x ATxxx.J9x.Qxx.8x Qxx.AKQ8.AT.AKxx 0.72200 
 2C-X-2N-P-P-P (8.8) 
J.xx.KJ98x.QJTxx Q9xx.KT9xx.x.Kxx 8xxx.J8x.Txxx.98 AKTx.AQx.AQx.Axx 0.88951 
 2C-X-2S-3C-3S-P-4N-P-5D-P-5H-P-6C-P-6S-P-P-P (12.1) 
J.xx.K

[{'call': 'X', 'insta_score': 0.694, 'expected_score': -139, 'adjustment': 35},
 {'call': 'PASS',
  'insta_score': 0.302,
  'expected_score': -169,
  'adjustment': 15}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


J.xx.KJ98x.QJTxx K98x.Txxx.xxx.9x ATxxx.J9x.Qxx.8x Qxx.AKQ8.AT.AKxx 0.72200 
 2C-X-2N-P-P-P (8.8)  
 2C-P-2N-P-3C-P-4N-P-5C-P-6C-P-P-P (6.9) 
J.xx.KJ98x.QJTxx Q9xx.KT9xx.x.Kxx 8xxx.J8x.Txxx.98 AKTx.AQx.AQx.Axx 0.88951 
 2C-X-2S-3C-3S-P-4N-P-5D-P-5H-P-6C-P-6S-P-P-P (12.1)  
 2C-P-2S-P-3S-P-4N-P-5C-P-5D-P-5N-P-6C-P-6H-P-6S-P-P-P (12.1) 
J.xx.KJ98x.QJTxx A9x.QJ98x.xxxx.9 xxxx.xxx.Tx.Kxxx KQT8x.AKT.AQ.A8x 0.84412 
 2C-X-2S-3C-P-P-3N-P-P-P (11.0)  
 2C-P-2S-P-P-P (12.1) 
J.xx.KJ98x.QJTxx Txxxx.AJ9xx.xx.x Q8x.T8xx.QTx.8xx AK9x.KQ.Axx.AK9x 0.93061 
 2C-X-2N-P-3D-P-4N-P-5C-P-5D-P-5N-P-6C-P-6D-P-6N-P-P-P (10.8)  
 2C-P-2N-P-3C-P-4N-P-5C-P-5N-P-6C-P-7C-P-P-P (7.7) 
J.xx.KJ98x.QJTxx T9x.AT9xx.xx.Kxx xxx.J8x.QTxx.8xx AKQ8xx.KQx.Ax.A9 0.98430 
 2C-X-2S-3C-P-P-3S-P-P-P (12.3)  
 2C-P-2S-P-P-P (12.3) 
J.xx.KJ98x.QJTxx 9xx.JT9xxx.ATx.8 QTxxx.8x.xxx.K9x AK8x.AKQ.Qx.Axxx 0.59018 
 2C-X-2N-P-3D-P-4N-P-5C-P-5D-P-5N-P-6C-P-6D-P-6N-P-P-P (10.4)  
 2C-P-2N-P-3D-P-4N-P-5C-P-5D-P-5N-P-6C-P-6D-P-6N-P-P-P (10.4)

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,0)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

sample_cards_auction, nsteps= 2
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', '1N', 'PASS', '3S', 'PASS']
nesw_i 0
Player:  N Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 8.98 15.06  8.15]]
Shape:  [[2.62 4.28 2.98 3.15 6.11 2.78 2.15 1.87 3.02 4.13 2.81 2.96]]
Setting seed (Sampling)= 640640594
sample_cards_vec generating 5000
sample_cards_vec took 0.107
n_samples 1237 matching bidding info
Samples after bidding filtering:  0  Threshold:  0.25
J.xx.KJ98x.QJTxx Q98x.Q8x.xx.Kxxx AKTxxx.AKx.AQx.A xx.JT9xx.Txx.98x 0.00
J.xx.KJ98x.QJTxx Txx.J8x.Txxx.K8x KQ8xxx.AKQx.A.A9 A9x.T9xx.Qxx.xxx 0.00
J.xx.KJ98x.QJTxx 8x.K9xx.xx.K9xxx AKQxxx.AQxx.Ax.A T9xx.JT8.QTxx.8x 0.00
J.xx.KJ98x.QJTxx xx.Q8xx.Qxx.K9xx AK8xxx.AKx.Axx.A QT9x.JT9x.Tx.8xx 0.00
J.xx.KJ98x.QJTxx K9x.J98.Txx.Axxx AQT8xx.AKQx.Ax.K xxx.Txxx.Qxx.98x 0.00
J.xx.KJ98x.QJTxx Kxx.T9x.Qxx.98xx AT98xx.AKQJ8..AK Qxx.xxx.ATxxx.xx 0.00
J.xx.KJ98x.QJTxx 8xx.KJxx.xx.K9xx AQT9xx.AQx.AQx.A Kxx.T98x.Txx.8xx 0.00
J.xx.KJ98x.QJTxx 8x.J